In [2]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import thomsonpy.thomson_scattering.thomson_scattering_tools as thtools
import thomsonpy.data_management.formatter as fmt
import thomsonpy.data_management.fragmenter as frm
import thomsonpy.data_management.octree.octree as octr
import thomsonpy.data_management.visualizer as vis
import thomsonpy.config.paths as paths
import thomsonpy.config.solar_imager_params as sip
import thomsonpy.config.thomson_scattering_params as tsp
import thomsonpy.constants.units as units

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
prefix = "../../"
predsci_data_filepath = prefix + paths.PREDSCI_DATA_PATH + paths.PREDSCI_FILENAME
data_filepaths = [f"{prefix}{paths.OCTREE_DATA_PATH}{os.path.splitext(paths.PREDSCI_FILENAME)[0]}_{i}.data" for i in range(4)]
octree_paths = [f"{prefix}{paths.OCTREES_PATH}octree_{i}.oct" for i in range(4)]
models_path = f"{prefix}{paths.MODELS_PATH}"
model_name = "p_allsun_6x6rsol_32s.np"

print(predsci_data_filepath)
print(data_filepaths)
print(octree_paths)
print(models_path)

../../data/predictive_science/eclipse2021_mhd_final/corona/rho002.hdf
['../../data/projects/cmag/data_format/rho002_0.data', '../../data/projects/cmag/data_format/rho002_1.data', '../../data/projects/cmag/data_format/rho002_2.data', '../../data/projects/cmag/data_format/rho002_3.data']
['../../data/projects/cmag/data_structures/octree_0.oct', '../../data/projects/cmag/data_structures/octree_1.oct', '../../data/projects/cmag/data_structures/octree_2.oct', '../../data/projects/cmag/data_structures/octree_3.oct']
../../data/projects/cmag/models/


In [4]:
# DATA FRAGMENTING
print("\nStarting data formatting and fragmenting...")

ne_raw = frm.get_ne_raw(predsci_data_filepath)
data_radial = frm.get_ne_raw_coords(predsci_data_filepath, "radial")
data_theta = frm.get_ne_raw_coords(predsci_data_filepath, "theta")
data_phi = frm.get_ne_raw_coords(predsci_data_filepath, "phi")

def selection1(r, theta, phi, ne):
    """
    First quadrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 5
    return (x <= limit and x >= 0 and
            y <= limit and y >= 0 and
            z <= limit / 2 and z >= -limit / 2)

def selection2(r, theta, phi, ne):
    """
    Second quadrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 5
    return (x <= 0 and x >= -limit and
            y <= limit and y >= 0 and
            z <= limit / 2 and z >= -limit / 2)
    
def selection3(r, theta, phi, ne):
    """
    Third quadrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 5
    return (x <= 0 and x >= -limit and
            y <= 0 and y >= -limit and
            z <= limit / 2 and z >= -limit / 2)
    
def selection4(r, theta, phi, ne):
    """
    Fourth quadrant
    """
    cartesian_coords = fmt.spherical_to_cartesian(r, theta, phi)
    x = cartesian_coords[0]
    y = cartesian_coords[1]
    z = cartesian_coords[2]
    limit = 5
    return (x <= limit and x >= 0 and
            y <= 0 and y >= -limit and
            z <= limit / 2 and z >= -limit / 2)
    
for i in range(0, 4):
    
    selection = None
    if i == 0:
        selection = selection1
    elif i == 1:
        selection = selection2
    elif i == 2:
        selection = selection3
    elif i == 3:
        selection = selection4
        
    ini_time = time.perf_counter()
    octree_data = []
    octree_data = frm.fragment(selection, fmt.apply_octree_data_format, ne_raw, data_radial, data_theta, data_phi)
    fin_time = time.perf_counter()

    print("Data formatting and fragmentation in", fin_time - ini_time, "seconds.")

    fmt.dump(data_filepaths[i], octree_data)
    print(f"Stored data for octree at {paths.OCTREE_DATA_PATH}")

    print("Octree will have", len(octree_data), "points")


Starting data formatting and fragmenting...

Loading datasets from ../../data/predictive_science/eclipse2021_mhd_final/corona/rho002.hdf :
 {'fakeDim0': (('fakeDim0',), (699,), 5, 0), 'fakeDim1': (('fakeDim1',), (327,), 5, 1), 'fakeDim2': (('fakeDim2',), (288,), 5, 2), 'Data-Set-2': (('fakeDim0', 'fakeDim1', 'fakeDim2'), (699, 327, 288), 5, 3)}
# POINTS =  65829024

Loading datasets from ../../data/predictive_science/eclipse2021_mhd_final/corona/rho002.hdf :
 {'fakeDim0': (('fakeDim0',), (699,), 5, 0), 'fakeDim1': (('fakeDim1',), (327,), 5, 1), 'fakeDim2': (('fakeDim2',), (288,), 5, 2), 'Data-Set-2': (('fakeDim0', 'fakeDim1', 'fakeDim2'), (699, 327, 288), 5, 3)}
RADIAL (RSOL, 1-30). # radial = 288

Loading datasets from ../../data/predictive_science/eclipse2021_mhd_final/corona/rho002.hdf :
 {'fakeDim0': (('fakeDim0',), (699,), 5, 0), 'fakeDim1': (('fakeDim1',), (327,), 5, 1), 'fakeDim2': (('fakeDim2',), (288,), 5, 2), 'Data-Set-2': (('fakeDim0', 'fakeDim1', 'fakeDim2'), (699, 327, 28

In [5]:
for i in range(0, 4):
    vis.vis_octree_data(data_filepaths[i])

In [4]:
import os
import time

import thomsonpy.config.octree_params as op

## OCTREE CREATION AND STORAGE
min_v = [op.MIN_1, op.MIN_2, op.MIN_3, op.MIN_4]
max_v = [op.MAX_1, op.MAX_2, op.MAX_3, op.MAX_4]

for i in range(0, 4):
    # Loading data...
    octree_data = fmt.load(data_filepaths[i])
    print("Loaded octree data:", len(octree_data), "points.")
    octree = 0
    # Creating octree...
    print("Creating octree with params:")
    print("MAX_LEVEL =", op.MAX_LEVEL)
    print("MAX_DATA =", op.MAX_DATA)
    print("MIN_V =", min_v[i])
    print("MAX_V =", max_v[i])
    ini_time = time.perf_counter()
    octree = octr.Octree(op.MAX_LEVEL, op.MAX_DATA, octree_data, min_v[i], max_v[i])
    fin_time = time.perf_counter()
    print("Octree " + str(i + 1) + " built in", str((fin_time - ini_time) / 60), "minutes.")
    octr.Octree.save(octree, octree_paths[i])
    print("Octree " + str(i + 1) + " created at", paths.OCTREES_PATH)

Loaded octree data: 12139242 points.
Creating octree with params:
MAX_LEVEL = 6
MAX_DATA = 1000
MIN_V = [ 0.00000e+00 -3.48170e+09 -1.74085e+09]
MAX_V = [3.48170e+09 0.00000e+00 1.74085e+09]
0.0 %
0.8237746640193845 %
1.647549328038769 %
2.4713239920581533 %
3.295098656077538 %
4.118873320096922 %
4.942647984116307 %
5.766422648135691 %
6.590197312155076 %
7.413971976174459 %
8.237746640193844 %
9.061521304213228 %
9.885295968232613 %
10.709070632251997 %
11.532845296271383 %
12.356619960290766 %
13.180394624310152 %
14.004169288329535 %
14.827943952348917 %
15.651718616368305 %
16.47549328038769 %
17.299267944407074 %
18.123042608426456 %
18.94681727244584 %
19.770591936465227 %
20.59436660048461 %
21.418141264503994 %
22.24191592852338 %
23.065690592542765 %
23.889465256562147 %
24.713239920581533 %
25.537014584600914 %
26.360789248620303 %
27.184563912639685 %
28.00833857665907 %
28.832113240678453 %
29.655887904697835 %
30.479662568717224 %
31.30343723273661 %
32.127211896755995 %


In [ ]:
for i in range (0, 4):
    vis.vis_octree(octr.Octree.load(octree_paths[i]))

In [8]:
"""
Starts execution of solar corona modeling with the Predictive Science model.
"""
IMAGE_NUM_POINTS = [576, 2321]
for i in IMAGE_NUM_POINTS:
    resolution = (sip.MAX_COORD - sip.MIN_COORD) / i
    print("Imaging between", sip.IMAGE_MIN * units.METERS_TO_RSOL, "RSol and" , sip.IMAGE_MAX * units.METERS_TO_RSOL, "RSol.")
    print("Resolution = ", resolution / 1000, " km (", i, "x", i, ").")
    x_values = np.linspace(sip.MIN_COORD, sip.MAX_COORD, i) # de - a +
    #print(x_values * units.METERS_TO_RSOL)
    y_values = np.linspace(sip.MIN_COORD, sip.MAX_COORD, i)[::-1] # de + a -
    #print(y_values * units.METERS_TO_RSOL)
    model = np.zeros((i, i))
    print("# Numeric integral steps =", tsp.NUM_Z)

    x_min = [i // 2, 0, 0, i // 2]
    print(x_min)
    x_max = [i, i // 2, i // 2, i]
    print(x_max)
    y_min = [0, 0, i // 2, i // 2]
    print(y_min)
    y_max = [i // 2, i // 2, i, i]
    print(y_max)
    num_points = i**2
    print(f"Num points = {num_points}")
    count = 0
    ini_time = time.perf_counter()
    for m in range(0, 4):
        print("Loading octree...")
        NE_MODEL = octr.Octree.load(octree_paths[m])
        print("Octree loaded.")
        for y in range(y_min[m], y_max[m]):
            for x in range(x_min[m], x_max[m]):
                if (x_values[x]**2 + y_values[y]**2) > tsp.SOLAR_RADIUS**2:
                    # Coordinates with the center of the Sun as Origin of the Reference System.
                    target = (x_values[x], y_values[y], 0)
                    # Creates a ThomsonGeometry object to manage the ray - tracing across the Corona.
                    TG = thtools.ThomsonGeometry(sip.SUN_CENTER, sip.OBSERVER, target, tsp.SOLAR_RADIUS)

                    # Line of sight integration generating a value for the scattered light model.
                    scattered_light = thtools.get_scattered_light(tsp.WAVELENGTH, tsp.T_SOL, tsp.X, TG.get_elongation(), tsp.INI_Z, tsp.FIN_Z, tsp.INCR_Z, TG, NE_MODEL)
                    model[y][x] = scattered_light
                else:
                    model[y][x] = 0

                # Progress
                count += 1
                if count % 10000 == 0:
                    print("Progress", count / num_points * 100,"%")
        
    fin_time = time.perf_counter()
    print("Model built in", str((fin_time - ini_time) / 60 / 60), "hours.")
    fmt.dump(f"{models_path}{i}{model_name}", model)

Imaging between [-3. -3.] RSol and [3. 3.] RSol.
Resolution =  1800.1034037052993  km ( 2321 x 2321 ).
# Numeric integral steps = 32.0
[1160, 0, 0, 1160]
[2321, 1160, 1160, 2321]
[0, 0, 1160, 1160]
[1160, 1160, 2321, 2321]
Num points = 5387041
Loading octree...
Octree loaded.
Progress 0.18563066440370513 %
Progress 0.37126132880741025 %
Progress 0.5568919932111154 %
Progress 0.7425226576148205 %
Progress 0.9281533220185255 %
Progress 1.1137839864222308 %
Progress 1.2994146508259359 %
Progress 1.485045315229641 %
Progress 1.670675979633346 %
Progress 1.856306644037051 %
Progress 2.0419373084407564 %
Progress 2.2275679728444615 %
Progress 2.4131986372481666 %
Progress 2.5988293016518718 %
Progress 2.784459966055577 %
Progress 2.970090630459282 %
Progress 3.155721294862987 %
Progress 3.341351959266692 %
Progress 3.5269826236703974 %
Progress 3.712613288074102 %
Progress 3.8982439524778076 %
Progress 4.083874616881513 %
Progress 4.269505281285218 %
Progress 4.455135945688923 %
Progress 4.6